In [44]:
import builtins
import os
import re
import glob
from pydantic import BaseModel

builtin_functions = [name for name in dir(builtins) if callable(getattr(builtins, name))]
print(builtin_functions)

['ArithmeticError', 'AssertionError', 'AttributeError', 'BaseException', 'BaseExceptionGroup', 'BlockingIOError', 'BrokenPipeError', 'BufferError', 'BytesWarning', 'ChildProcessError', 'ConnectionAbortedError', 'ConnectionError', 'ConnectionRefusedError', 'ConnectionResetError', 'DeprecationWarning', 'EOFError', 'EncodingWarning', 'EnvironmentError', 'Exception', 'ExceptionGroup', 'FileExistsError', 'FileNotFoundError', 'FloatingPointError', 'FutureWarning', 'GeneratorExit', 'IOError', 'ImportError', 'ImportWarning', 'IndentationError', 'IndexError', 'InterruptedError', 'IsADirectoryError', 'KeyError', 'KeyboardInterrupt', 'LookupError', 'MemoryError', 'ModuleNotFoundError', 'NameError', 'NotADirectoryError', 'NotImplementedError', 'OSError', 'OverflowError', 'PendingDeprecationWarning', 'PermissionError', 'ProcessLookupError', 'RecursionError', 'ReferenceError', 'ResourceWarning', 'RuntimeError', 'RuntimeWarning', 'StopAsyncIteration', 'StopIteration', 'SyntaxError', 'SyntaxWarning', 

In [6]:
import re

def get_used_functions(script_path):
    with open(script_path, 'r') as file:
        content = file.read()

    # Regular expression to find function calls
    # This looks for any characters (non-greedy) followed by an opening parenthesis,
    # capturing everything after the last whitespace or start of the line
    pattern = r'(^|[\s])([^\s]+?)\s*\('

    # Find all matches
    matches = re.findall(pattern, content, re.MULTILINE)

    # Extract the full function names (second group in each match)
    used_functions = set(match[1] for match in matches)

    return used_functions

In [ ]:
import re

def get_used_functions_with_indent(script_path):
    with open(script_path, 'r') as file:
        lines = file.readlines()

    function_info = []
    
    for line_number, line in enumerate(lines, 1):
        # Skip lines that start with '#' (comments)
        if line.lstrip().startswith('#'):
            continue

        # Count leading spaces to determine indent level
        indent_level = len(line) - len(line.lstrip())
        
        # Regular expression to find function calls
        # This looks for alphanumeric characters and dots, followed immediately by an opening parenthesis
        matches = re.finditer(r'\b([a-zA-Z0-9_\.]+)\s*\(', line)
        
        for match in matches:
            function_name = match.group(1)
            # Ensure the function name doesn't end with a dot
            if not function_name.endswith('.'):
                function_info.append({
                    'name': function_name,
                    'line': line_number,
                    'indent': indent_level
                })

    return function_info


# Example usage
script_path = 'C:/Users/justl/Documents/folder/main.py'
functions = get_used_functions_with_indent(script_path)

print("Functions used in the script with their indent levels:")
for func in functions:
    print(f"  Line {func['line']}: {' ' * func['indent']}{func['name']} (indent: {func['indent']})")


In [ ]:
folder = "C:/Users/justl/Documents/folder/"
entry_point = "main.py"

start_file_path = os.path.join(folder, entry_point)


In [ ]:
import re
import json

class Node:
    def __init__(self, type, content, children=None):
        self.type = type
        self.content = content
        self.children = children or []

def parse_python_to_nodes(script_path):
    with open(script_path, 'r') as file:
        lines = file.readlines()

    root = Node(type='root', '')
    stack = [root]
    current_indent = 0

    for line in lines:
        line = line.rstrip()
        if line.strip() == '' or line.lstrip().startswith('#'):
            continue

        indent = len(line) - len(line.lstrip())
        content = line.strip()

        while indent < current_indent:
            stack.pop()
            current_indent -= 4

        if content.startswith('#'):
            node = Node(type='comment', content)
        elif content.startswith('def '):
            node = Node(type='function', content)
        elif content.startswith('if ') or content.startswith('elif ') or content.startswith('else:'):
            node = Node(type='condition', content)
        elif content.startswith('for ') or content.startswith('while '):
            node = Node(type='loop', content)
        elif content.startswith('import ') or content.startswith('from '):
            node = Node(type='import', content)
        elif '(' in content:
            node = Node(type='function_call', content)
        else:
            node = Node(type='statement', content)

        stack[-1].children.append(node)
        
        if content.endswith(':'):
            stack.append(node)
            current_indent = indent + 4

    return root

def node_to_dict(node):
    return {
        'type': node.type,
        'content': node.content,
        'children': [node_to_dict(child) for child in node.children]
    }

# Example usage
script_path = 'C:/Users/justl/Documents/folder/main.py'
root = parse_python_to_nodes(script_path)

# Convert to JSON for easy visualization or further processing
json_representation = json.dumps(node_to_dict(root), indent=2)
print(json_representation)


In [121]:
import json
import re
from typing import *

class Alias(BaseModel):
    actual_name: str
    alias_name: str
    local_file: bool

class Node(BaseModel):
    type: str
    content: str
    children: List['Node'] = []
    functions: List[str] = []
    aliases: List[Alias] = []

# class Node:
#     def __init__(self, type, content, children=None, function=None):
#         self.type = type
#         self.content = content
#         self.children = children or []
#         self.function = function

def get_alias(node: Node, folder: str):
    filename = node.content.split(" ")[1]
    filename = filename.replace(".", "/")

    file_loc = os.path.join(folder, filename+".py")

    local_file = os.path.exists(file_loc)

    statement = node.content
    aliases = []

    if " as " in statement:
        parts = statement.split()
        indexes = [i for i, part in enumerate(parts) if part == "as"]
        for index in indexes:
            actual_name = parts[index - 1]  # The name before "as"
            alias_name = parts[index + 1]    # The name after "as"

            alias = Alias(
                alias_name = alias_name,
                actual_name = actual_name,
                local_file = local_file
            )

            aliases.append(alias)

    return aliases

def alias_to_dict(alias: Alias) -> dict:
    return {
        'actual_name': alias.actual_name,
        'alias_name': alias.alias_name,
        'local_file': alias.local_file
    }

def node_to_dict(node):
    return {
        'type': node.type,
        'content': node.content,
        'functions': node.functions,
        'children': [node_to_dict(child) for child in node.children],
        'aliases': [alias_to_dict(alias) for alias in node.aliases if alias],
    }

def dict_to_node(data: Dict[str, Any]) -> Node:
    # Create the Node object from the dictionary
    node = Node(
        type=data['type'],
        content=data['content'],
        function=data.get('function'),
        alias=Alias(**data['alias']) if data.get('alias') else None
    )
    
    # Recursively convert children
    for child_data in data.get('children', []):
        child_node = dict_to_node(child_data)
        node.children.append(child_node)
    
    return node

In [ ]:
import os
import re
import json
from typing import List

class Alias(BaseModel):
    actual_name: str
    alias_name: str
    local_file: bool

class Node(BaseModel):
    type: str
    content: str
    children: List['Node'] = []
    functions: List[str] = []  # List of function names
    aliases: List[Alias] = []   # List of Alias objects

def get_alias(node: Node, folder: str):
    filename = node.content.split(" ")[1]
    filename = filename.replace(".", "/")
    file_loc = os.path.join(folder, filename + ".py")
    local_file = os.path.exists(file_loc)

    statement = node.content
    aliases = []

    # Split by commas to handle both single and multiline imports
    parts = statement.replace("(", "").replace(")", "").split(",")
    for part in parts:
        part = part.strip()
        if " as " in part:
            actual_name, alias_name = part.split(" as ")
            alias = Alias(
                alias_name=alias_name.strip(),
                actual_name=actual_name.strip(),
                local_file=local_file
            )
            aliases.append(alias)
            node.aliases.append(alias)  # Add alias to the Node's aliases list

    return aliases

def extract_function_name(line):
    match = re.findall(r'\b([a-zA-Z0-9_\.]+)\s*\(', line)
    return match if match else []  # Return a list of function names

def handle_class(stripped_line, stack):
    if 'BaseModel' in stripped_line:
        node = Node(type='custom_type', content=stripped_line)
    else:
        node = Node(type='class', content=stripped_line)
    stack[-1].children.append(node)
    stack.append(node)

def handle_function(stripped_line, stack):
    function_names = extract_function_name(stripped_line)
    node = Node(type='function', content=stripped_line, functions=function_names)
    stack[-1].children.append(node)
    stack.append(node)

def handle_condition(stripped_line, stack):
    node = Node(type='condition', content=stripped_line)
    stack[-1].children.append(node)
    if stripped_line.endswith(':'):
        stack.append(node)

def handle_loop(stripped_line, stack):
    node = Node(type='loop', content=stripped_line)
    stack[-1].children.append(node)
    if stripped_line.endswith(':'):
        stack.append(node)

def handle_import(stripped_line, stack, script_folder, output_folder):
    node = Node(type='import', content=stripped_line)
    stack[-1].children.append(node)
    aliases = []

    # Extract aliases from the import statement
    tmp_aliases = get_alias(node, script_folder)
    if tmp_aliases:
        aliases.extend(tmp_aliases)

    # Handle the filename and local file check
    filename = node.content.split(" ")[1].replace(".", "/")
    file_loc = os.path.join(script_folder, filename + ".py")
    local_file = os.path.exists(file_loc)

    if local_file:
        parse_python_to_nodes(script_folder, filename + ".py", output_folder)

    return aliases

def parse_python_to_nodes(script_folder, script_name, output_folder="nodes"):
    script_path = os.path.join(script_folder, script_name)

    filename, _ = os.path.splitext(script_name)
    filename = filename.replace("/", ".").replace("\\", ".")

    with open(script_path, 'r') as file:
        lines = file.readlines()

    if not os.path.exists(output_folder):
        os.mkdir(output_folder)

    node_file = os.path.join(output_folder, filename + ".json")

    root = Node(type='root', content='')
    stack = [root]
    current_indent = 0
    in_docstring = False
    docstring_content = []
    aliases = []

    def extract_function_name(line):
        match = re.findall(r'\b([a-zA-Z0-9_\.]+)\s*\(', line)
        return match if match else []  # Return a list of function names

    for line in lines:
        stripped_line = line.strip()
        indent = len(line) - len(line.lstrip())

        # Handle docstrings
        if '"""' in stripped_line:
            if not in_docstring:
                in_docstring = True
                docstring_content = [stripped_line]
            else:
                docstring_content.append(stripped_line)
                full_docstring = '\n'.join(docstring_content)
                node = Node(type='docstring', content=full_docstring)
                if stack:  # Ensure stack is not empty
                    stack[-1].children.append(node)
                in_docstring = False
                docstring_content = []
            continue

        if in_docstring:
            docstring_content.append(line.rstrip())
            continue

        if stripped_line == '' or stripped_line.startswith('#'):
            continue

        # Adjust stack based on indentation
        while stack and indent < current_indent:
            stack.pop()
            current_indent -= 4

        # Reset stack if it becomes empty
        if not stack:
            stack.append(root)  # Reset to root
            current_indent = 0

        # Handle definitions with a mapping
        line_first_word = stripped_line.split()[0]
        handlers = {
            'class': handle_class,
            'def': handle_function,
            'if': handle_condition,
            'elif': handle_condition,
            'else': handle_condition,
            'for': handle_loop,
            'while': handle_loop,
            'import': handle_import,
            'from': handle_import
        }

        if line_first_word in handlers:
            if line_first_word in ['import', 'from']:
                handlers[line_first_word](stripped_line, stack, script_folder, output_folder)
            else:
                handlers[line_first_word](stripped_line, stack, output_folder)

        elif '(' in stripped_line:
            function_names = extract_function_name(stripped_line)
            node = Node(type='function_call', content=stripped_line, functions=function_names)
            
            if function_names == []:
                node = Node(type='statement', content=stripped_line)
                stack[-1].children.append(node)
                continue
            
            for alias in aliases:
                for function in function_names:
                    if alias.alias_name in function.split("."):
                        node.aliases.append(alias)

            stack[-1].children.append(node)
        else:
            node = Node(type='statement', content=stripped_line)
            stack[-1].children.append(node)

    data = node_to_dict(root)
    with open(node_file, 'w') as json_file:
        json.dump(data, json_file, indent=4)

    return root

In [135]:
def parse_python_to_nodes(script_folder, script_name, output_folder="nodes"):
    script_path = os.path.join(script_folder, script_name)

    filename, _ = os.path.splitext(script_name)
    filename = filename.replace("/", ".")
    filename = filename.replace("\\", ".")

    with open(script_path, 'r') as file:
        lines = file.readlines()

    if not os.path.exists(output_folder):
        os.mkdir(output_folder)

    node_file = os.path.join(output_folder, filename + ".json")
    # if os.path.exists(node_file):
    #     return False

    root = Node(type='root', content='')
    stack = [root]
    current_indent = 0
    in_docstring = False
    docstring_content = []
    aliases = []

    def extract_function_name(line):
        match = re.findall(r'\b([a-zA-Z0-9_\.]+)\s*\(', line)
        return match if match else []  # Return a list of function names

    for line in lines:
        stripped_line = line.strip()
        indent = len(line) - len(line.lstrip())

        # Handle docstrings
        if '"""' in stripped_line:
            if not in_docstring:
                in_docstring = True
                docstring_content = [stripped_line]
            else:
                docstring_content.append(stripped_line)
                full_docstring = '\n'.join(docstring_content)
                node = Node(type='docstring', content=full_docstring)
                if stack:  # Ensure stack is not empty
                    stack[-1].children.append(node)
                in_docstring = False
                docstring_content = []
            continue

        if in_docstring:
            docstring_content.append(line.rstrip())
            continue

        if stripped_line == '' or stripped_line.startswith('#'):
            continue

        # Adjust stack based on indentation
        while stack and indent < current_indent:
            stack.pop()
            current_indent -= 4

        # Reset stack if it becomes empty
        if not stack:
            stack.append(root)  # Reset to root
            current_indent = 0

        # Handle definitions
        if stripped_line.startswith('class '):
            if 'BaseModel' in stripped_line:
                node = Node(type='custom_type', content=stripped_line)  # Set type to custom_type
            else:
                node = Node(type='class', content=stripped_line)  # Set type to class
            stack[-1].children.append(node)
            stack.append(node)  # Push the class node onto the stack
            current_indent = indent + 4

        elif stripped_line.startswith('def '):
            function_names = extract_function_name(stripped_line)
            node = Node(type='function', content=stripped_line, functions=function_names)
            stack[-1].children.append(node)
            stack.append(node)
            current_indent = indent + 4

        elif stripped_line.startswith('if ') or stripped_line.startswith('elif ') or stripped_line.startswith('else:'):
            node = Node(type='condition', content=stripped_line)
            stack[-1].children.append(node)
            if stripped_line.endswith(':'):
                stack.append(node)
                current_indent = indent + 4

        elif stripped_line.startswith('for ') or stripped_line.startswith('while '):
            node = Node(type='loop', content=stripped_line)
            stack[-1].children.append(node)
            if stripped_line.endswith(':'):
                stack.append(node)
                current_indent = indent + 4
                
        elif stripped_line.startswith('import ') or stripped_line.startswith('from '):
            node = Node(type='import', content=stripped_line)
            stack[-1].children.append(node)

            # Extract aliases from the import statement
            tmp_aliases = get_alias(node, script_folder)
            if tmp_aliases:
                aliases.extend(tmp_aliases)

            # Handle the filename and local file check
            filename = node.content.split(" ")[1]
            filename = filename.replace(".", "/")

            file_loc = os.path.join(script_folder, filename + ".py")

            local_file = os.path.exists(file_loc)

            if local_file:
                # Call parse_python_to_nodes only if the file exists
                parse_python_to_nodes(script_folder, filename + ".py", output_folder)

        elif '(' in stripped_line:
            function_names = extract_function_name(stripped_line)
            node = Node(type='function_call', content=stripped_line, functions=function_names)
            if function_names == []:
                node = Node(type='statement', content=stripped_line)
                stack[-1].children.append(node)
                continue
            
            for alias in aliases:
                for function in function_names:
                    if alias.alias_name in function.split("."):
                        node.aliases.append(alias)

            stack[-1].children.append(node)
        else:
            node = Node(type='statement', content=stripped_line)
            stack[-1].children.append(node)

    data = node_to_dict(root)
    with open(node_file, 'w') as json_file:
        json.dump(data, json_file, indent=4)

    return True

def collect_functions_and_classes(output_folder):
    # Dictionary to hold the results
    functions_dict = {}

    # Get all JSON files in the output folder
    json_files = glob.glob(os.path.join(output_folder, "*.json"))
    
    for json_file in json_files:
        # Extract the filename without the extension
        filename = os.path.splitext(os.path.basename(json_file))[0]
        
        with open(json_file, 'r') as file:
            data = json.load(file)

        root_node = dict_to_node(data)

        # Initialize a list to hold function names for this file
        function_list = []

        # Recursive function to traverse the nodes
        def traverse_nodes(node):
            # Add functions
            if node.type == 'function':
                method_name_start = node.content.split()[1]
                method_name = method_name_start.split("(")[0]
                function_list.append(method_name)  # Get the function name

            # Add class methods
            if node.type == 'class':
                class_name = node.content.split()[1].split(':')[0]  # Get the class name
                for child in node.children:
                    if child.type == 'function':
                        method_name_start = child.content.split()[1]  # Get the method name
                        method_name = method_name_start.split("(")[0]
                        function_list.append(f"{class_name}.{method_name}")

            # Traverse children
            for child in node.children:
                traverse_nodes(child)

        # Start traversing from the root node
        traverse_nodes(root_node)

        # Store the collected functions in the dictionary
        functions_dict[filename] = function_list

    return functions_dict


def main(script_path, script_name):
    parse_python_to_nodes(script_path, script_name, output_folder="nodes")
    x = collect_functions_and_classes(output_folder="nodes")
    for key in x.keys():
        print(key, x[key])

script_path = 'C:/Users/justl/Documents/VideoDiarization'
script_name = "main_v3.py"
root = main(script_path, script_name)

# Convert to JSON for easy visualization or further processing
# json_representation = json.dumps(node_to_dict(root), indent=2)
# print(json_representation)


audio_v2 ['AudioDiarizationProcessor.__init__', 'AudioDiarizationProcessor.process_chunk', 'AudioDiarizationProcessor.save_chunk_state', 'AudioDiarizationProcessor.build_segments', 'AudioDiarizationProcessor.ms_to_timecode', 'AudioDiarizationProcessor.timecode_to_ms', 'AudioDiarizationProcessor.build_segments_by_words', '__init__', 'process_chunk', 'save_chunk_state', 'build_segments', 'ms_to_timecode', 'timecode_to_ms', 'build_segments_by_words', 'build_segments_by_sentences', 'AudioDiarization.__init__', 'AudioDiarization.create_transcription', 'AudioDiarization.remove_background_audio', 'AudioDiarization.locate_silences', 'AudioDiarization.get_audio_duration_ms', 'AudioDiarization.ensure_standard_format', 'AudioDiarization.split_audio', 'AudioDiarization.create_audio_chunks', 'AudioDiarization.plot_audio_db', 'AudioDiarization.extract_transcription', 'AudioDiarization.run_audio_diarization', 'AudioDiarization.transcribeAudioFile', 'AudioDiarization.withRetry', '__init__', 'create_tr

In [26]:
def find_import_nodes(node):
    import_nodes = []
    
    if node.type == 'import':
        import_nodes.append(node)
    
    for child in node.children:
        import_nodes.extend(find_import_nodes(child))
    
    return import_nodes

# Assuming you have already created the 'root' variable
import_nodes = find_import_nodes(root)

In [ ]:
class Alias()

In [55]:
original_dir = "C:/Users/justl/Documents/folder"

for import_node in import_nodes:
    statement = import_node.content

    filename = statement.split(" ")[1]
    filename = filename.replace(".", "/")
    file_loc = os.path.join(original_dir, filename+".py")

    local_file = os.path.exists(file_loc)

    aliases = []

    if " as " in statement:
        parts = statement.split()
        indexes = [i for i, part in enumerate(parts) if part == "as"]
        for index in indexes:
            actual_name = parts[index - 1]  # The name before "as"
            alias_name = parts[index + 1]    # The name after "as"

            alias = Alias(
                alias_name = alias_name,
                actual_name = actual_name,
                local_file = local_file
            )

            aliases.append(alias)

    # if "as" in parts:
    #     splits = statement.split(" as ")
    #     print(splits)
        
    # if statement.startswith("import "):
    #     for part in parts[1:]:
    #         if "as" in parts:
    #             alias = part.split(" as ")
    #             print(alias)
                
    

np numpy


In [58]:
statement = "from numpy import x as y, a as b, w"

if " as " in statement:
    parts = statement.split()
    indexes = [i for i, part in enumerate(parts) if part == "as"]
    for index in indexes:
        actual_name = parts[index - 1]  # The name before "as"
        alias_name = parts[index + 1]    # The name after "as"

        print(alias_name, actual_name)


y, x
b, a


In [158]:
import ast



In [166]:
import ast

def get_functions_classes_and_imports_from_file(script_folder, script_name):
    file_path = os.path.join(script_folder, script_name)

    with open(file_path, "r") as file:
        # Parse the file into an AST
        node = ast.parse(file.read(), filename=file_path)

    functions = []
    classes = []
    imports = []

    for item in node.body:
        # print(type(item))
        if isinstance(item, ast.FunctionDef):
            print(ast.get_docstring(item))
            functions.append(item.name)  # Add the function name

        elif isinstance(item, ast.ClassDef):
            classes.append(item.name)  # Add the class name
            # Collect methods within the class
            for method in item.body:
                if isinstance(method, ast.FunctionDef):
                    functions.append(f"{item.name}.{method.name}")  # Class method

        elif isinstance(item, ast.Import):
            for alias in item.names:
                imports.append(alias.name)  # Add the imported module name

        elif isinstance(item, ast.ImportFrom):
            # Handle imports from specific modules
            module = item.module if item.module else ""
            for alias in item.names:
                imports.append(f"{module}.{alias.name}")  # Add the specific import

    return functions, classes, imports

# Example usage
script_folder = "C:/Users/justl/Documents/VideoDiarization"  # Replace with your actual Python file path
script_name = "main_v3.py"

functions, classes, imports = get_functions_classes_and_imports_from_file(script_folder, script_name)

for import_name in imports:
    module_loc = import_name.split(".")
    if len(module_loc) > 1:
        module_loc = module_loc[:-1]
        
    module_loc = "/".join(module_loc)
    
    module_path = os.path.join(script_folder, module_loc + ".py")
    
    if os.path.exists(module_path):
        print(module_path, True)
    else:
        print(module_path, False)

print("Functions:", functions)
print("Classes:", classes)
print("Imports:", imports)


# # Example usage
# functions, classes = get_functions_and_classes_from_file(file_path)

# print("Functions:", functions)
# print("Classes:", classes)

Main script for the diarization of an mp4 video file.
asfoiawfeshjnikpwlnifawlnikafwjlbolia
C:/Users/justl/Documents/VideoDiarization\click.py False
C:/Users/justl/Documents/VideoDiarization\os.py False
C:/Users/justl/Documents/VideoDiarization\typing.py False
C:/Users/justl/Documents/VideoDiarization\time.py False
C:/Users/justl/Documents/VideoDiarization\glob.py False
C:/Users/justl/Documents/VideoDiarization\logging.py False
C:/Users/justl/Documents/VideoDiarization\numpy.py False
C:/Users/justl/Documents/VideoDiarization\custom_types/config.py True
C:/Users/justl/Documents/VideoDiarization\custom_types/vision.py True
C:/Users/justl/Documents/VideoDiarization\custom_types/vision.py True
C:/Users/justl/Documents/VideoDiarization\custom_types/vision.py True
C:/Users/justl/Documents/VideoDiarization\custom_types/audio.py True
C:/Users/justl/Documents/VideoDiarization\custom_types/audio.py True
C:/Users/justl/Documents/VideoDiarization\custom_types/video.py True
C:/Users/justl/Documents

In [169]:
class ASTNode(BaseModel):
    type: str
    content: str
    children: List['Node'] = []
    functions: List[str] = []
    aliases: List[Alias] = []

In [168]:
import ast
import importlib.util
import sys

# List of standard library modules
standard_library_modules = {
    'sys', 'os', 'math', 'json', 're', 'datetime', 'collections', 'itertools', 'functools', 'numpy', 'pandas', 'requests'
    # Add more standard library modules as needed
}

def is_standard_library(module_name):
    """Check if the given module is a standard library module."""
    return module_name in standard_library_modules

class CodeVisitor(ast.NodeVisitor):
    def __init__(self):
        self.functions = []
        self.classes = []
        self.imports = []
        self.function_calls = []

    def visit_FunctionDef(self, node):
        self.functions.append(node.name)
        self.generic_visit(node)

    def visit_ClassDef(self, node):
        self.classes.append(node.name)
        for item in node.body:
            if isinstance(item, ast.FunctionDef):
                self.functions.append(f"{node.name}.{item.name}")
        self.generic_visit(node)

    def visit_Import(self, node):
        for alias in node.names:
            module_name = alias.name
            if is_standard_library(module_name):
                self.imports.append((module_name, "library"))
            else:
                self.imports.append((module_name, "custom"))
        self.generic_visit(node)

    def visit_ImportFrom(self, node):
        module = node.module if node.module else ""
        for alias in node.names:
            if is_standard_library(module):
                self.imports.append((f"{module}.{alias.name}", "library"))
            else:
                self.imports.append((f"{module}.{alias.name}", "custom"))
        self.generic_visit(node)

    def visit_Call(self, node):
        if isinstance(node.func, ast.Name):
            self.function_calls.append(node.func.id)
        elif isinstance(node.func, ast.Attribute):
            self.function_calls.append(f"{self.get_full_attribute_name(node.func)}")
        self.generic_visit(node)

    def get_full_attribute_name(self, node):
        if isinstance(node.value, ast.Name):
            return f"{node.value.id}.{node.attr}"
        elif isinstance(node.value, ast.Attribute):
            return f"{self.get_full_attribute_name(node.value)}.{node.attr}"
        return node.attr

def get_functions_classes_imports_and_calls_from_file(file_path):
    with open(file_path, "r") as file:
        # Parse the file into an AST
        node = ast.parse(file.read(), filename=file_path)

    visitor = CodeVisitor()
    visitor.visit(node)

    return visitor.functions, visitor.classes, visitor.imports, visitor.function_calls

# Example usage
file_path = "C:/Users/justl/Documents/VideoDiarization/main_v3.py"  # Replace with your actual Python file path
functions, classes, imports, function_calls = get_functions_classes_imports_and_calls_from_file(file_path)

print("Function Definitions:", functions)
print("Classes:", classes)
print("Imports:", imports)
print("Function Calls:", function_calls)


Function Definitions: ['main']
Classes: []
Imports: [('click', 'custom'), ('os', 'library'), ('typing.*', 'custom'), ('time', 'custom'), ('glob', 'custom'), ('logging', 'custom'), ('numpy', 'library'), ('custom_types.config.Config', 'custom'), ('custom_types.vision.FaceLocation', 'custom'), ('custom_types.vision.Frame', 'custom'), ('custom_types.vision.TalkNetOutput', 'custom'), ('custom_types.audio.AudioChunk', 'custom'), ('custom_types.audio.Stage1Output', 'custom'), ('custom_types.video.VideoSegment', 'custom'), ('vision_v2.VisionDiarization', 'custom'), ('audio_v2.AudioDiarization', 'custom'), ('audio_v2.AudioDiarizationProcessor', 'custom'), ('video_v2.VideoDiarization', 'custom'), ('talknet.run_talknet', 'custom'), ('talknet.load_talknet_output', 'custom'), ('utils.setup_project', 'custom'), ('utils.mp4_to_wav', 'custom')]
Function Calls: ['logging.basicConfig', 'logging.getLogger', 'setup_project', 'os.path.join', 'logger.setLevel', 'logger.setLevel', 'os.path.splitext', 'logger

In [193]:
import ast
import json
from typing import List, Optional
from pydantic import BaseModel

class Alias(BaseModel):
    actual_name: str
    alias_name: str
    local_file: bool

class Node(BaseModel):
    type: str
    content: str
    children: List['Node'] = []
    function: Optional[str] = None
    alias: Optional[Alias] = None
    line_number: Optional[int] = None
    end_line_number: Optional[int] = None

class PyFileAnalyzer(ast.NodeVisitor):
    def __init__(self, source_code, file_path, save_folder):
        self.root = Node(type="module", content="", children=[])
        self.current_node = self.root
        self.save_folder = save_folder
        self.file_path = file_path
        self.folder_path = os.path.dirname(file_path)
        self.source_code = source_code.splitlines()

    def get_node_content(self, node):
        start_line = node.lineno - 1
        end_line = node.end_lineno
        content = ' '.join(self.source_code[start_line:end_line]).strip()
        return content

    def visit_FunctionDef(self, node):
        content = self.get_node_content(node)
        func_node = Node(
            type="function",
            content=content,
            line_number=node.lineno,
            end_line_number=node.end_lineno,
            function=node.name
        )
        self.current_node.children.append(func_node)
        old_node = self.current_node
        self.current_node = func_node
        self.generic_visit(node)
        self.current_node = old_node

    def visit_ClassDef(self, node):
        content = self.get_node_content(node)
        class_node = Node(
            type="class",
            content=content,
            line_number=node.lineno,
            end_line_number=node.end_lineno
        )
        self.current_node.children.append(class_node)
        old_node = self.current_node
        self.current_node = class_node
        self.generic_visit(node)
        self.current_node = old_node

    def visit_Import(self, node):
        content = self.get_node_content(node)
        for alias in node.names:
            ### local file check
            tmp_local_file_path = alias.name
            tmp_abs_file_path = os.path.join(self.folder_path, tmp_local_file_path + ".py")

            if os.path.exists(tmp_abs_file_path):
                local_file = True
                analyze_python_file(tmp_abs_file_path, self.save_folder)
            else:
                local_file = False

            import_node = Node(
                type="import",
                content=content,
                line_number=node.lineno,
                alias=Alias(actual_name=alias.name, alias_name=alias.asname or alias.name, local_file=local_file)
            )
            self.current_node.children.append(import_node)

    def visit_ImportFrom(self, node):
        content = self.get_node_content(node)
        module = node.module or ""
        
        tmp_local_file_path = f"{module}".replace(".", "/")
        tmp_abs_file_path = os.path.join(self.folder_path, tmp_local_file_path + ".py")

        if os.path.exists(tmp_abs_file_path):
            local_file = True
            analyze_python_file(tmp_abs_file_path, self.save_folder)
        else:
            local_file = False

        for alias in node.names:
            import_node = Node(
                type="import",
                content=content,
                line_number=node.lineno,
                alias=Alias(actual_name=f"{module}.{alias.name}", alias_name=alias.asname or alias.name, local_file=local_file)
            )
            self.current_node.children.append(import_node)

    def visit_Call(self, node):
        content = self.get_node_content(node)
        if isinstance(node.func, ast.Name):
            call_node = Node(
                type="function_call",
                content=content,
                line_number=node.lineno,
                function=node.func.id
            )
            self.current_node.children.append(call_node)
        elif isinstance(node.func, ast.Attribute):
            call_node = Node(
                type="function_call",
                content=content,
                line_number=node.lineno,
                function=self.get_full_attribute_name(node.func)
            )
            self.current_node.children.append(call_node)
        self.generic_visit(node)

    def get_full_attribute_name(self, node):
        if isinstance(node.value, ast.Name):
            return f"{node.value.id}.{node.attr}"
        elif isinstance(node.value, ast.Attribute):
            return f"{self.get_full_attribute_name(node.value)}.{node.attr}"
        return node.attr

def analyze_python_file(file_path: str, save_folder: str) -> Node:
    file_path = os.path.abspath(file_path)

    file_name = os.path.basename(file_path)
    file_name = os.path.splitext(file_name)[0]

    json_path = os.path.join(save_folder, file_name + ".json")

    with open(file_path, "r") as file:
        source_code = file.read()
        tree = ast.parse(source_code, filename=file_path)
    
    analyzer = PyFileAnalyzer(source_code, file_path, save_folder)
    analyzer.visit(tree)

    root_node = analyzer.root

    json_output = json.dumps(node_to_dict(root_node), indent=2)

    with open(json_path, "w") as json_file:
        json_file.write(json_output)

    return analyzer.root

def node_to_dict(node: Node) -> dict:
    result = {
        "type": node.type,
        "content": node.content,
        "children": [node_to_dict(child) for child in node.children],
        "line_number": node.line_number,
        "end_line_number": node.end_line_number
    }
    if node.function:
        result["function"] = node.function
    if node.alias:
        result["alias"] = node.alias.dict()
    return result


# Example usage
file_path = "C:/Users/justl/Documents/VideoDiarization/main_v3.py"  # Replace with your actual Python file path
root_node = analyze_python_file(file_path, save_folder="test_folder")

# # Convert to JSON
# json_output = json.dumps(node_to_dict(root_node), indent=2)

# # Save to file
# with open("test_folder/output.json", "w") as json_file:
#     json_file.write(json_output)

print("Analysis complete. Results saved to output.json")


Analysis complete. Results saved to output.json


In [181]:
file_path = "C:/Users/justl/Documents/VideoDiarization/main_v3.py"
file_path = os.path.abspath(file_path)

folder_path = os.path.dirname(file_path)

with open(file_path, "r") as file:
    # Parse the file into an AST
    node = ast.parse(file.read(), filename=file_path)


for item in node.body:
    if isinstance(item, ast.Import) or isinstance(item, ast.ImportFrom):
        

In [200]:
import ast
import json
import os
from typing import List, Optional, Dict
from pydantic import BaseModel

class Alias(BaseModel):
    actual_name: str
    alias_name: str
    local_file: bool

class Node(BaseModel):
    type: str
    content: str
    children: List['Node'] = []
    function: Optional[str] = None
    alias: Optional[Alias] = None
    line_number: Optional[int] = None
    end_line_number: Optional[int] = None
    docstring: Optional[str] = None
    return_type: Optional[str] = None

class PyFileAnalyzer(ast.NodeVisitor):
    def __init__(self, source_code, file_path, save_folder):
        self.root = Node(type="module", content="", children=[])
        self.current_node = self.root
        self.save_folder = save_folder
        self.file_path = file_path
        self.folder_path = os.path.dirname(file_path)
        self.source_code = source_code.splitlines()
        self.alias_dict = {}
        self.variable_types = {}

    def get_node_content(self, node):
        start_line = node.lineno - 1
        end_line = node.end_lineno
        content = ' '.join(self.source_code[start_line:end_line]).strip()
        return content

    def visit_ImportFrom(self, node):
        content = self.get_node_content(node)
        module = node.module or ""
        
        for alias in node.names:
            new_alias = Alias(
                actual_name=f"{module}.{alias.name}", 
                alias_name=alias.asname or alias.name, 
                local_file=True  # Assuming it's a local file for simplicity
            )
            self.alias_dict[alias.asname or alias.name] = new_alias

            import_node = Node(
                type="import",
                content=content,
                line_number=node.lineno,
                alias=new_alias
            )
            self.current_node.children.append(import_node)

    def visit_AnnAssign(self, node):
        if isinstance(node.target, ast.Name) and isinstance(node.annotation, ast.Name):
            var_name = node.target.id
            type_name = node.annotation.id
            if type_name in self.alias_dict:
                self.variable_types[var_name] = self.alias_dict[type_name].actual_name

    def visit_Call(self, node):
        content = self.get_node_content(node)
        if isinstance(node.func, ast.Name):
            function_name = node.func.id
            if function_name in self.alias_dict:
                function_name = self.alias_dict[function_name].actual_name
            call_node = Node(
                type="function_call",
                content=content,
                line_number=node.lineno,
                function=function_name
            )
            self.current_node.children.append(call_node)
        elif isinstance(node.func, ast.Attribute):
            full_name = self.get_full_attribute_name(node.func)
            parts = full_name.split('.')
            if parts[0] in self.variable_types:
                parts[0] = self.variable_types[parts[0]]
            function_name = '.'.join(parts)
            call_node = Node(
                type="function_call",
                content=content,
                line_number=node.lineno,
                function=function_name
            )
            self.current_node.children.append(call_node)
        self.generic_visit(node)

    def get_full_attribute_name(self, node):
        if isinstance(node.value, ast.Name):
            return f"{node.value.id}.{node.attr}"
        elif isinstance(node.value, ast.Attribute):
            return f"{self.get_full_attribute_name(node.value)}.{node.attr}"
        return node.attr
    
    
def analyze_python_file(file_path: str, save_folder: str) -> Node:
    file_path = os.path.abspath(file_path)
    file_name = os.path.basename(file_path)
    json_path = os.path.join(save_folder, f"{file_name}.json")

    try:
        with open(file_path, "r") as file:
            source_code = file.read()
            tree = ast.parse(source_code, filename=file_path)
        
        analyzer = PyFileAnalyzer(source_code, file_path, save_folder)
        analyzer.visit(tree)

        root_node = analyzer.root

        json_output = json.dumps(node_to_dict(root_node), indent=2)

        with open(json_path, "w") as json_file:
            json_file.write(json_output)

        print(f"Analyzed and saved JSON for: {file_path}")
        return analyzer.root
    except Exception as e:
        print(f"Error analyzing file {file_path}: {str(e)}")
        return None

def node_to_dict(node: Node) -> dict:
    result = {
        "type": node.type,
        "content": node.content,
        "children": [node_to_dict(child) for child in node.children],
        "line_number": node.line_number,
        "end_line_number": node.end_line_number
    }
    if node.function:
        result["function"] = node.function
    if node.alias:
        result["alias"] = node.alias.dict()
    if node.docstring:
        result["docstring"] = node.docstring
    if node.return_type:
        result["return_type"] = node.return_type
    return result

def map_execution(file_path: str, save_folder: str):
    os.makedirs(save_folder, exist_ok=True)
    analyze_python_file(file_path, save_folder)

# Example usage
file_path = "C:/Users/justl/Documents/VideoDiarization/main_v3.py"  # Replace with your actual Python file path
save_folder = "analysis_output"
map_execution(file_path, save_folder)


Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\main_v3.py


In [202]:
import ast
import json
import os
from typing import List, Optional, Dict
from pydantic import BaseModel

class Alias(BaseModel):
    actual_name: str
    alias_name: str
    local_file: bool

class Node(BaseModel):
    type: str
    content: str
    children: List['Node'] = []
    function: Optional[str] = None
    alias: Optional[Alias] = None
    line_number: Optional[int] = None
    end_line_number: Optional[int] = None
    docstring: Optional[str] = None
    return_type: Optional[str] = None

class PyFileAnalyzer(ast.NodeVisitor):
    def __init__(self, source_code, file_path, save_folder):
        self.root = Node(type="module", content="", children=[])
        self.current_node = self.root
        self.save_folder = save_folder
        self.file_path = file_path
        self.folder_path = os.path.dirname(file_path)
        self.source_code = source_code.splitlines()
        self.alias_dict = {}
        self.variable_types = {}

    def get_node_content(self, node):
        start_line = node.lineno - 1
        end_line = node.end_lineno
        content = ' '.join(self.source_code[start_line:end_line]).strip()
        return content

    def visit_Import(self, node):
        content = self.get_node_content(node)
        for alias in node.names:
            tmp_local_file_path = alias.name.replace('.', os.path.sep) + ".py"
            tmp_abs_file_path = os.path.join(self.folder_path, tmp_local_file_path)

            local_file = os.path.exists(tmp_abs_file_path)

            if local_file:
                analyze_python_file(tmp_abs_file_path, self.save_folder)

            new_alias = Alias(actual_name=alias.name, alias_name=alias.asname or alias.name, local_file=local_file)
            self.alias_dict[alias.asname or alias.name] = new_alias

            import_node = Node(
                type="import",
                content=content,
                line_number=node.lineno,
                alias=new_alias
            )
            self.current_node.children.append(import_node)

    def visit_ImportFrom(self, node):
        content = self.get_node_content(node)
        module = node.module or ""
        
        tmp_local_file_path = module.replace('.', os.path.sep) + ".py"
        tmp_abs_file_path = os.path.join(self.folder_path, tmp_local_file_path)

        local_file = os.path.exists(tmp_abs_file_path)

        if local_file:
            analyze_python_file(tmp_abs_file_path, self.save_folder)

        for alias in node.names:
            new_alias = Alias(
                actual_name=f"{module}.{alias.name}", 
                alias_name=alias.asname or alias.name, 
                local_file=local_file
            )
            self.alias_dict[alias.asname or alias.name] = new_alias

            import_node = Node(
                type="import",
                content=content,
                line_number=node.lineno,
                alias=new_alias
            )
            self.current_node.children.append(import_node)

    def visit_AnnAssign(self, node):
        if isinstance(node.target, ast.Name) and isinstance(node.annotation, ast.Name):
            var_name = node.target.id
            type_name = node.annotation.id
            if type_name in self.alias_dict:
                self.variable_types[var_name] = self.alias_dict[type_name].actual_name

    def visit_Call(self, node):
        content = self.get_node_content(node)
        if isinstance(node.func, ast.Name):
            function_name = node.func.id
            if function_name in self.alias_dict:
                function_name = self.alias_dict[function_name].actual_name
            call_node = Node(
                type="function_call",
                content=content,
                line_number=node.lineno,
                function=function_name
            )
            self.current_node.children.append(call_node)
        elif isinstance(node.func, ast.Attribute):
            full_name = self.get_full_attribute_name(node.func)
            parts = full_name.split('.')
            if parts[0] in self.variable_types:
                parts[0] = self.variable_types[parts[0]]
            function_name = '.'.join(parts)
            call_node = Node(
                type="function_call",
                content=content,
                line_number=node.lineno,
                function=function_name
            )
            self.current_node.children.append(call_node)
        self.generic_visit(node)

    def get_full_attribute_name(self, node):
        if isinstance(node.value, ast.Name):
            return f"{node.value.id}.{node.attr}"
        elif isinstance(node.value, ast.Attribute):
            return f"{self.get_full_attribute_name(node.value)}.{node.attr}"
        return node.attr

def analyze_python_file(file_path: str, save_folder: str) -> Node:
    file_path = os.path.abspath(file_path)
    file_name = os.path.basename(file_path)
    file_name = os.path.splitext(file_path)[0]
    json_path = os.path.join(save_folder, f"{file_name}.json")

    # Check if the file has already been analyzed
    if os.path.exists(json_path):
        print(f"Skipping already analyzed file: {file_path}")
        return None

    try:
        with open(file_path, "r") as file:
            source_code = file.read()
            tree = ast.parse(source_code, filename=file_path)
        
        analyzer = PyFileAnalyzer(source_code, file_path, save_folder)
        analyzer.visit(tree)

        root_node = analyzer.root

        json_output = json.dumps(node_to_dict(root_node), indent=2)

        with open(json_path, "w") as json_file:
            json_file.write(json_output)

        print(f"Analyzed and saved JSON for: {file_path}")
        return analyzer.root
    except Exception as e:
        print(f"Error analyzing file {file_path}: {str(e)}")
        return None

def node_to_dict(node: Node) -> dict:
    result = {
        "type": node.type,
        "content": node.content,
        "children": [node_to_dict(child) for child in node.children],
        "line_number": node.line_number,
        "end_line_number": node.end_line_number
    }
    if node.function:
        result["function"] = node.function
    if node.alias:
        result["alias"] = node.alias.dict()
    if node.docstring:
        result["docstring"] = node.docstring
    if node.return_type:
        result["return_type"] = node.return_type
    return result

def map_execution(file_path: str, save_folder: str):
    os.makedirs(save_folder, exist_ok=True)
    analyze_python_file(file_path, save_folder)

    print(f"Execution map complete. Results saved to {save_folder}")

# Example usage
file_path = "C:/Users/justl/Documents/VideoDiarization/main_v3.py"
save_folder = "analysis_output"
map_execution(file_path, save_folder)


Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\config.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\vision.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\audio.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\video.py
Skipping already analyzed file: C:\Users\justl\Documents\VideoDiarization\custom_types\config.py
Skipping already analyzed file: C:\Users\justl\Documents\VideoDiarization\custom_types\vision.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\vision_v2.py
Skipping already analyzed file: C:\Users\justl\Documents\VideoDiarization\custom_types\config.py
Skipping already analyzed file: C:\Users\justl\Documents\VideoDiarization\custom_types\audio.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\constants.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\

In [203]:
import ast
import importlib
import os
from typing import List, Optional, Dict
from pydantic import BaseModel

class Alias(BaseModel):
    actual_name: str
    alias_name: str
    local_file: bool

class Node(BaseModel):
    type: str
    content: str
    children: List['Node'] = []
    function: Optional[str] = None
    alias: Optional[Alias] = None
    line_number: Optional[int] = None
    end_line_number: Optional[int] = None
    docstring: Optional[str] = None
    return_type: Optional[str] = None

class PyFileAnalyzer(ast.NodeVisitor):
    def __init__(self, source_code, file_path, save_folder):
        self.root = Node(type="module", content="", children=[])
        self.current_node = self.root
        self.save_folder = save_folder
        self.file_path = file_path
        self.folder_path = os.path.dirname(file_path)
        self.source_code = source_code.splitlines()
        self.alias_dict = {}
        self.variable_types = {}
        self.analyzed_files = set()

    def get_node_content(self, node):
        start_line = node.lineno - 1
        end_line = node.end_lineno
        content = ' '.join(self.source_code[start_line:end_line]).strip()
        return content

    def visit_Import(self, node):
        content = self.get_node_content(node)
        for alias in node.names:
            tmp_local_file_path = alias.name.replace('.', os.path.sep) + ".py"
            tmp_abs_file_path = os.path.join(self.folder_path, tmp_local_file_path)

            local_file = os.path.exists(tmp_abs_file_path)

            if local_file and tmp_abs_file_path not in self.analyzed_files:
                self.analyzed_files.add(tmp_abs_file_path)
                analyze_python_file(tmp_abs_file_path, self.save_folder)

            new_alias = Alias(actual_name=alias.name, alias_name=alias.asname or alias.name, local_file=local_file)
            self.alias_dict[alias.asname or alias.name] = new_alias

            import_node = Node(
                type="import",
                content=content,
                line_number=node.lineno,
                alias=new_alias
            )
            self.current_node.children.append(import_node)

    def visit_ImportFrom(self, node):
        content = self.get_node_content(node)
        module = node.module or ""
        
        tmp_local_file_path = module.replace('.', os.path.sep) + ".py"
        tmp_abs_file_path = os.path.join(self.folder_path, tmp_local_file_path)

        local_file = os.path.exists(tmp_abs_file_path)

        if local_file and tmp_abs_file_path not in self.analyzed_files:
            self.analyzed_files.add(tmp_abs_file_path)
            analyze_python_file(tmp_abs_file_path, self.save_folder)

        if len(node.names) == 1 and node.names[0].name == '*':
            # Handle star import
            self.handle_star_import(module, tmp_abs_file_path if local_file else None)
        else:
            for alias in node.names:
                new_alias = Alias(
                    actual_name=f"{module}.{alias.name}", 
                    alias_name=alias.asname or alias.name, 
                    local_file=local_file
                )
                self.alias_dict[alias.asname or alias.name] = new_alias

                import_node = Node(
                    type="import",
                    content=content,
                    line_number=node.lineno,
                    alias=new_alias
                )
                self.current_node.children.append(import_node)

    def handle_star_import(self, module_name, module_path):
        if module_path:
            # For local modules
            with open(module_path, 'r') as file:
                module_ast = ast.parse(file.read())
            names = self.get_module_names(module_ast)
        else:
            # For non-local modules
            try:
                module = importlib.import_module(module_name)
                names = dir(module)
            except ImportError:
                print(f"Warning: Unable to import module {module_name}")
                return

        for name in names:
            if not name.startswith('_'):  # Exclude private names
                new_alias = Alias(
                    actual_name=f"{module_name}.{name}",
                    alias_name=name,
                    local_file=bool(module_path)
                )
                self.alias_dict[name] = new_alias

    def get_module_names(self, module_ast):
        names = []
        for node in module_ast.body:
            if isinstance(node, (ast.FunctionDef, ast.ClassDef)):
                names.append(node.name)
        return names

    def visit_AnnAssign(self, node):
        if isinstance(node.target, ast.Name) and isinstance(node.annotation, ast.Name):
            var_name = node.target.id
            type_name = node.annotation.id
            if type_name in self.alias_dict:
                self.variable_types[var_name] = self.alias_dict[type_name].actual_name

    def visit_Call(self, node):
        content = self.get_node_content(node)
        if isinstance(node.func, ast.Name):
            function_name = node.func.id
            if function_name in self.alias_dict:
                function_name = self.alias_dict[function_name].actual_name
            call_node = Node(
                type="function_call",
                content=content,
                line_number=node.lineno,
                function=function_name
            )
            self.current_node.children.append(call_node)
        elif isinstance(node.func, ast.Attribute):
            full_name = self.get_full_attribute_name(node.func)
            parts = full_name.split('.')
            if parts[0] in self.variable_types:
                parts[0] = self.variable_types[parts[0]]
            function_name = '.'.join(parts)
            call_node = Node(
                type="function_call",
                content=content,
                line_number=node.lineno,
                function=function_name
            )
            self.current_node.children.append(call_node)
        self.generic_visit(node)

    def get_full_attribute_name(self, node):
        if isinstance(node.value, ast.Name):
            return f"{node.value.id}.{node.attr}"
        elif isinstance(node.value, ast.Attribute):
            return f"{self.get_full_attribute_name(node.value)}.{node.attr}"
        return node.attr

def analyze_python_file(file_path: str, save_folder: str) -> Node:
    file_path = os.path.abspath(file_path)
    file_name = os.path.basename(file_path)
    json_path = os.path.join(save_folder, f"{file_name}.json")

    # Check if the file has already been analyzed
    # if os.path.exists(json_path):
    #     print(f"Skipping already analyzed file: {file_path}")
    #     return None

    try:
        with open(file_path, "r") as file:
            source_code = file.read()
            tree = ast.parse(source_code, filename=file_path)
        
        analyzer = PyFileAnalyzer(source_code, file_path, save_folder)
        analyzer.visit(tree)

        root_node = analyzer.root

        json_output = json.dumps(node_to_dict(root_node), indent=2)

        with open(json_path, "w") as json_file:
            json_file.write(json_output)

        print(f"Analyzed and saved JSON for: {file_path}")
        return analyzer.root
    except Exception as e:
        print(f"Error analyzing file {file_path}: {str(e)}")
        return None

def node_to_dict(node: Node) -> dict:
    result = {
        "type": node.type,
        "content": node.content,
        "children": [node_to_dict(child) for child in node.children],
        "line_number": node.line_number,
        "end_line_number": node.end_line_number
    }
    if node.function:
        result["function"] = node.function
    if node.alias:
        result["alias"] = node.alias.dict()
    if node.docstring:
        result["docstring"] = node.docstring
    if node.return_type:
        result["return_type"] = node.return_type
    return result

def map_execution(file_path: str, save_folder: str):
    os.makedirs(save_folder, exist_ok=True)
    analyze_python_file(file_path, save_folder)

    print(f"Execution map complete. Results saved to {save_folder}")

# Example usage
file_path = "C:/Users/justl/Documents/VideoDiarization/main_v3.py"  # Replace with your actual Python file path
save_folder = "analysis_output"
map_execution(file_path, save_folder)


Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\config.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\vision.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\audio.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\video.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\config.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\vision.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\vision_v2.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\config.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\audio.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\constants.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\audio_v2.py


In [204]:
import ast
import json
import importlib
import os
from typing import List, Optional, Dict
from pydantic import BaseModel

class Alias(BaseModel):
    actual_name: str
    alias_name: str
    local_file: bool

class Node(BaseModel):
    type: str
    content: str
    children: List['Node'] = []
    function: Optional[str] = None
    alias: Optional[Alias] = None
    line_number: Optional[int] = None
    end_line_number: Optional[int] = None
    docstring: Optional[str] = None
    return_type: Optional[str] = None
    indent_level: int = 0

class PyFileAnalyzer(ast.NodeVisitor):
    def __init__(self, source_code, file_path, save_folder):
        self.root = Node(type="module", content="", children=[])
        self.current_node = self.root
        self.save_folder = save_folder
        self.file_path = file_path
        self.folder_path = os.path.dirname(file_path)
        self.source_code = source_code.splitlines()
        self.alias_dict = {}
        self.variable_types = {}
        self.analyzed_files = set()
        self.indent_level = 0

    def get_node_content(self, node):
        start_line = node.lineno - 1
        end_line = node.end_lineno
        content = '\n'.join(self.source_code[start_line:end_line]).strip()
        return content

    def visit_Import(self, node):
        content = self.get_node_content(node)
        for alias in node.names:
            tmp_local_file_path = alias.name.replace('.', os.path.sep) + ".py"
            tmp_abs_file_path = os.path.join(self.folder_path, tmp_local_file_path)

            local_file = os.path.exists(tmp_abs_file_path)

            if local_file and tmp_abs_file_path not in self.analyzed_files:
                self.analyzed_files.add(tmp_abs_file_path)
                analyze_python_file(tmp_abs_file_path, self.save_folder)

            new_alias = Alias(actual_name=alias.name, alias_name=alias.asname or alias.name, local_file=local_file)
            self.alias_dict[alias.asname or alias.name] = new_alias

            import_node = Node(
                type="import",
                content=content,
                line_number=node.lineno,
                end_line_number=node.end_lineno,
                alias=new_alias,
                indent_level=self.indent_level
            )
            self.current_node.children.append(import_node)

    def visit_ImportFrom(self, node):
        content = self.get_node_content(node)
        module = node.module or ""
        
        tmp_local_file_path = module.replace('.', os.path.sep) + ".py"
        tmp_abs_file_path = os.path.join(self.folder_path, tmp_local_file_path)

        local_file = os.path.exists(tmp_abs_file_path)

        if local_file and tmp_abs_file_path not in self.analyzed_files:
            self.analyzed_files.add(tmp_abs_file_path)
            analyze_python_file(tmp_abs_file_path, self.save_folder)

        if len(node.names) == 1 and node.names[0].name == '*':
            self.handle_star_import(module, tmp_abs_file_path if local_file else None)
        else:
            for alias in node.names:
                new_alias = Alias(
                    actual_name=f"{module}.{alias.name}", 
                    alias_name=alias.asname or alias.name, 
                    local_file=local_file
                )
                self.alias_dict[alias.asname or alias.name] = new_alias

                import_node = Node(
                    type="import",
                    content=content,
                    line_number=node.lineno,
                    end_line_number=node.end_lineno,
                    alias=new_alias,
                    indent_level=self.indent_level
                )
                self.current_node.children.append(import_node)

    def handle_star_import(self, module_name, module_path):
        if module_path:
            with open(module_path, 'r') as file:
                module_ast = ast.parse(file.read())
            names = self.get_module_names(module_ast)
        else:
            try:
                module = importlib.import_module(module_name)
                names = dir(module)
            except ImportError:
                print(f"Warning: Unable to import module {module_name}")
                return

        for name in names:
            if not name.startswith('_'):
                new_alias = Alias(
                    actual_name=f"{module_name}.{name}",
                    alias_name=name,
                    local_file=bool(module_path)
                )
                self.alias_dict[name] = new_alias

    def get_module_names(self, module_ast):
        names = []
        for node in module_ast.body:
            if isinstance(node, (ast.FunctionDef, ast.ClassDef)):
                names.append(node.name)
        return names

    def visit_AnnAssign(self, node):
        if isinstance(node.target, ast.Name) and isinstance(node.annotation, ast.Name):
            var_name = node.target.id
            type_name = node.annotation.id
            if type_name in self.alias_dict:
                self.variable_types[var_name] = self.alias_dict[type_name].actual_name

    def visit_Call(self, node):
        content = self.get_node_content(node)
        if isinstance(node.func, ast.Name):
            function_name = node.func.id
            if function_name in self.alias_dict:
                function_name = self.alias_dict[function_name].actual_name
            call_node = Node(
                type="function_call",
                content=content,
                line_number=node.lineno,
                end_line_number=node.end_lineno,
                function=function_name,
                indent_level=self.indent_level
            )
            self.current_node.children.append(call_node)
        elif isinstance(node.func, ast.Attribute):
            full_name = self.get_full_attribute_name(node.func)
            parts = full_name.split('.')
            if parts[0] in self.variable_types:
                parts[0] = self.variable_types[parts[0]]
            function_name = '.'.join(parts)
            call_node = Node(
                type="function_call",
                content=content,
                line_number=node.lineno,
                end_line_number=node.end_lineno,
                function=function_name,
                indent_level=self.indent_level
            )
            self.current_node.children.append(call_node)
        self.generic_visit(node)

    def get_full_attribute_name(self, node):
        if isinstance(node.value, ast.Name):
            return f"{node.value.id}.{node.attr}"
        elif isinstance(node.value, ast.Attribute):
            return f"{self.get_full_attribute_name(node.value)}.{node.attr}"
        return node.attr

    def visit_For(self, node):
        content = self.get_node_content(node)
        for_node = Node(
            type="for",
            content=content,
            line_number=node.lineno,
            end_line_number=node.end_lineno,
            indent_level=self.indent_level
        )
        self.current_node.children.append(for_node)
        self.current_node = for_node
        self.indent_level += 1
        self.generic_visit(node)
        self.indent_level -= 1
        self.current_node = self.current_node.children[-1]

    def visit_While(self, node):
        content = self.get_node_content(node)
        while_node = Node(
            type="while",
            content=content,
            line_number=node.lineno,
            end_line_number=node.end_lineno,
            indent_level=self.indent_level
        )
        self.current_node.children.append(while_node)
        self.current_node = while_node
        self.indent_level += 1
        self.generic_visit(node)
        self.indent_level -= 1
        self.current_node = self.current_node.children[-1]

    def visit_If(self, node):
        content = self.get_node_content(node)
        if_node = Node(
            type="if",
            content=content,
            line_number=node.lineno,
            end_line_number=node.end_lineno,
            indent_level=self.indent_level
        )
        self.current_node.children.append(if_node)
        self.current_node = if_node
        self.indent_level += 1
        self.generic_visit(node)
        self.indent_level -= 1
        self.current_node = self.current_node.children[-1]

def analyze_python_file(file_path: str, save_folder: str) -> Node:
    file_path = os.path.abspath(file_path)
    file_name = os.path.basename(file_path)
    json_path = os.path.join(save_folder, f"{file_name}.json")

    # if os.path.exists(json_path):
    #     print(f"Skipping already analyzed file: {file_path}")
    #     return None

    try:
        with open(file_path, "r") as file:
            source_code = file.read()
            tree = ast.parse(source_code, filename=file_path)
        
        analyzer = PyFileAnalyzer(source_code, file_path, save_folder)
        analyzer.visit(tree)

        root_node = analyzer.root

        json_output = json.dumps(node_to_dict(root_node), indent=2)

        with open(json_path, "w") as json_file:
            json_file.write(json_output)

        print(f"Analyzed and saved JSON for: {file_path}")
        return analyzer.root
    except Exception as e:
        print(f"Error analyzing file {file_path}: {str(e)}")
        return None

def node_to_dict(node: Node) -> dict:
    result = {
        "type": node.type,
        "content": node.content,
        "children": [node_to_dict(child) for child in node.children],
        "line_number": node.line_number,
        "end_line_number": node.end_line_number,
        "indent_level": node.indent_level
    }
    if node.function:
        result["function"] = node.function
    if node.alias:
        result["alias"] = node.alias.dict()
    if node.docstring:
        result["docstring"] = node.docstring
    if node.return_type:
        result["return_type"] = node.return_type
    return result

def map_execution(file_path: str, save_folder: str):
    os.makedirs(save_folder, exist_ok=True)
    analyze_python_file(file_path, save_folder)

    print(f"Execution map complete. Results saved to {save_folder}")

# Example usage
file_path = "C:/Users/justl/Documents/VideoDiarization/main_v3.py"  # Replace with your actual Python file path
save_folder = "analysis_output"
map_execution(file_path, save_folder)


Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\config.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\vision.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\audio.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\video.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\config.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\vision.py
Error analyzing file C:\Users\justl\Documents\VideoDiarization\vision_v2.py: list index out of range
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\config.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\audio.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\constants.py
Error analyzing file C:\Users\justl\Documents\VideoDiarization\aud

In [210]:
import ast
import json
import importlib
import os
from typing import List, Optional, Dict
from pydantic import BaseModel

class Alias(BaseModel):
    actual_name: str
    alias_name: str
    local_file: bool

class Node(BaseModel):
    type: str
    content: str
    children: List['Node'] = []
    function: Optional[str] = None
    alias: Optional[Alias] = None
    line_number: Optional[int] = None
    end_line_number: Optional[int] = None
    docstring: Optional[str] = None
    return_type: Optional[str] = None
    indent_level: int = 0

class PyFileAnalyzer(ast.NodeVisitor):
    def __init__(self, source_code, file_path, save_folder, global_function_defs):
        self.root = Node(type="module", content="", children=[])
        self.current_node = self.root
        self.save_folder = save_folder
        self.file_path = file_path
        self.folder_path = os.path.dirname(file_path)
        self.source_code = source_code.splitlines()
        self.alias_dict = {}
        self.variable_types = {}
        self.analyzed_files = set()
        self.indent_level = 0
        self.function_defs = {}
        self.current_function_stack = []
        self.global_function_defs = global_function_defs

    def get_node_content(self, node):
        start_line = node.lineno - 1
        end_line = node.end_lineno
        content = '\n'.join(self.source_code[start_line:end_line]).strip()
        return content

    def visit_Import(self, node):
        content = self.get_node_content(node)
        for alias in node.names:
            tmp_local_file_path = alias.name.replace('.', os.path.sep) + ".py"
            tmp_abs_file_path = os.path.join(self.folder_path, tmp_local_file_path)

            local_file = os.path.exists(tmp_abs_file_path)

            if local_file and tmp_abs_file_path not in self.analyzed_files:
                self.analyzed_files.add(tmp_abs_file_path)
                analyze_python_file(tmp_abs_file_path, self.save_folder, self.global_function_defs)

            new_alias = Alias(actual_name=alias.name, alias_name=alias.asname or alias.name, local_file=local_file)
            self.alias_dict[alias.asname or alias.name] = new_alias

            import_node = Node(
                type="import",
                content=content,
                line_number=node.lineno,
                end_line_number=node.end_lineno,
                alias=new_alias,
                indent_level=self.indent_level
            )
            self.current_node.children.append(import_node)

    def visit_ImportFrom(self, node):
        content = self.get_node_content(node)
        module = node.module or ""
        
        tmp_local_file_path = module.replace('.', os.path.sep) + ".py"
        tmp_abs_file_path = os.path.join(self.folder_path, tmp_local_file_path)

        local_file = os.path.exists(tmp_abs_file_path)

        if local_file and tmp_abs_file_path not in self.analyzed_files:
            self.analyzed_files.add(tmp_abs_file_path)
            analyze_python_file(tmp_abs_file_path, self.save_folder, self.global_function_defs)

        if len(node.names) == 1 and node.names[0].name == '*':
            self.handle_star_import(module, tmp_abs_file_path if local_file else None)
        else:
            for alias in node.names:
                new_alias = Alias(
                    actual_name=f"{module}.{alias.name}", 
                    alias_name=alias.asname or alias.name, 
                    local_file=local_file
                )
                self.alias_dict[alias.asname or alias.name] = new_alias

                import_node = Node(
                    type="import",
                    content=content,
                    line_number=node.lineno,
                    end_line_number=node.end_lineno,
                    alias=new_alias,
                    indent_level=self.indent_level
                )
                self.current_node.children.append(import_node)

    def handle_star_import(self, module_name, module_path):
        if module_path:
            with open(module_path, 'r') as file:
                module_ast = ast.parse(file.read())
            names = self.get_module_names(module_ast)
        else:
            try:
                module = importlib.import_module(module_name)
                names = dir(module)
            except ImportError:
                print(f"Warning: Unable to import module {module_name}")
                return

        for name in names:
            if not name.startswith('_'):
                new_alias = Alias(
                    actual_name=f"{module_name}.{name}",
                    alias_name=name,
                    local_file=bool(module_path)
                )
                self.alias_dict[name] = new_alias

    def get_module_names(self, module_ast):
        names = []
        for node in module_ast.body:
            if isinstance(node, (ast.FunctionDef, ast.ClassDef)):
                names.append(node.name)
        return names

    def visit_FunctionDef(self, node):
        content = self.get_node_content(node)
        func_node = Node(
            type="function_def",
            content=content,
            line_number=node.lineno,
            end_line_number=node.end_lineno,
            function=node.name,
            indent_level=self.indent_level,
            docstring=ast.get_docstring(node)
        )
        self.function_defs[node.name] = func_node
        self.global_function_defs[f"{os.path.basename(self.file_path)}:{node.name}"] = func_node
        self.current_node.children.append(func_node)
        
        old_node = self.current_node
        self.current_node = func_node
        self.indent_level += 1
        self.current_function_stack.append(node.name)
        self.generic_visit(node)
        self.current_function_stack.pop()
        self.indent_level -= 1
        self.current_node = old_node

    def visit_AnnAssign(self, node):
        if isinstance(node.target, ast.Name) and isinstance(node.annotation, ast.Name):
            var_name = node.target.id
            type_name = node.annotation.id
            if type_name in self.alias_dict:
                self.variable_types[var_name] = self.alias_dict[type_name].actual_name

    def visit_Call(self, node):
        content = self.get_node_content(node)
        if isinstance(node.func, ast.Name):
            function_name = node.func.id
            if function_name in self.alias_dict:
                function_name = self.alias_dict[function_name].actual_name
        elif isinstance(node.func, ast.Attribute):
            full_name = self.get_full_attribute_name(node.func)
            parts = full_name.split('.')
            if parts[0] in self.variable_types:
                parts[0] = self.variable_types[parts[0]]
            function_name = '.'.join(parts)
        else:
            function_name = "unknown"

        call_node = Node(
            type="function_call",
            content=content,
            line_number=node.lineno,
            end_line_number=node.end_lineno,
            function=function_name,
            indent_level=self.indent_level
        )
        self.current_node.children.append(call_node)

        for full_func_name, func_def in self.global_function_defs.items():
            if full_func_name.endswith(f":{function_name.split('.')[-1]}") and function_name not in self.current_function_stack:
                func_def_copy = func_def.copy(deep=True)
                func_def_copy.indent_level = self.indent_level + 1
                call_node.children.append(func_def_copy)
                break

        self.generic_visit(node)

    def get_full_attribute_name(self, node):
        if isinstance(node.value, ast.Name):
            return f"{node.value.id}.{node.attr}"
        elif isinstance(node.value, ast.Attribute):
            return f"{self.get_full_attribute_name(node.value)}.{node.attr}"
        return node.attr

    def visit_For(self, node):
        content = self.get_node_content(node)
        for_node = Node(
            type="for",
            content=content,
            line_number=node.lineno,
            end_line_number=node.end_lineno,
            indent_level=self.indent_level
        )
        self.current_node.children.append(for_node)
        self.current_node = for_node
        self.indent_level += 1
        self.generic_visit(node)
        self.indent_level -= 1
        self.current_node = self.current_node.children[-1]

    def visit_While(self, node):
        content = self.get_node_content(node)
        while_node = Node(
            type="while",
            content=content,
            line_number=node.lineno,
            end_line_number=node.end_lineno,
            indent_level=self.indent_level
        )
        self.current_node.children.append(while_node)
        self.current_node = while_node
        self.indent_level += 1
        self.generic_visit(node)
        self.indent_level -= 1
        self.current_node = self.current_node.children[-1]

    def visit_If(self, node):
        content = self.get_node_content(node)
        if_node = Node(
            type="if",
            content=content,
            line_number=node.lineno,
            end_line_number=node.end_lineno,
            indent_level=self.indent_level
        )
        self.current_node.children.append(if_node)
        self.current_node = if_node
        self.indent_level += 1
        self.generic_visit(node)
        self.indent_level -= 1
        self.current_node = self.current_node.children[-1]

def analyze_python_file(file_path: str, save_folder: str, global_function_defs: Dict[str, Node]) -> Node:
    file_path = os.path.abspath(file_path)
    file_name = os.path.basename(file_path)
    json_path = os.path.join(save_folder, f"{file_name}.json")

    # if os.path.exists(json_path):
    #     print(f"Skipping already analyzed file: {file_path}")
    #     return None

    try:
        with open(file_path, "r") as file:
            source_code = file.read()
            tree = ast.parse(source_code, filename=file_path)
        
        analyzer = PyFileAnalyzer(source_code, file_path, save_folder, global_function_defs)
        analyzer.visit(tree)

        root_node = analyzer.root

        json_output = json.dumps(node_to_dict(root_node), indent=2)

        with open(json_path, "w") as json_file:
            json_file.write(json_output)

        print(f"Analyzed and saved JSON for: {file_path}")
        return analyzer.root
    except Exception as e:
        print(f"Error analyzing file {file_path}: {str(e)}")
        return None

def node_to_dict(node: Node) -> dict:
    result = {
        "type": node.type,
        "content": node.content,
        "children": [node_to_dict(child) for child in node.children],
        "line_number": node.line_number,
        "end_line_number": node.end_line_number,
        "indent_level": node.indent_level
    }
    if node.function:
        result["function"] = node.function
    if node.alias:
        result["alias"] = node.alias.dict()
    if node.docstring:
        result["docstring"] = node.docstring
    if node.return_type:
        result["return_type"] = node.return_type
    return result

def map_execution(file_path: str, save_folder: str):
    os.makedirs(save_folder, exist_ok=True)
    global_function_defs = {}
    analyze_python_file(file_path, save_folder, global_function_defs)

    print(f"Execution map complete. Results saved to {save_folder}")

file_path = "C:/Users/justl/Documents/VideoDiarization/main_v3.py"
save_folder = "analysis_output"
map_execution(file_path, save_folder)


Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\config.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\vision.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\audio.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\video.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\config.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\vision.py
Error analyzing file C:\Users\justl\Documents\VideoDiarization\vision_v2.py: list index out of range
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\config.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\custom_types\audio.py
Analyzed and saved JSON for: C:\Users\justl\Documents\VideoDiarization\constants.py
Error analyzing file C:\Users\justl\Documents\VideoDiarization\aud

In [ ]:
class Alias(BaseModel):
    actual_name: str
    alias_name: str
    local_file: bool

class Node(BaseModel):
    type: str
    content: str
    children: List['Node'] = []
    function: Optional[str] = None
    alias: Optional[Alias] = None
    line_number: Optional[int] = None
    end_line_number: Optional[int] = None
    docstring: Optional[str] = None
    return_type: Optional[str] = None
    indent_level: int = 0
    def_location: Optional[str] = None

class PyFileAnalyzer(ast.NodeVisitor):
    def __init__(self, source_code, file_path, save_folder, global_function_defs):
        self.root = Node(type="module", content="", children=[])
        self.current_node = self.root

        self.save_folder = save_folder
        self.file_path = file_path
        self.folder_path = os.path.dirname(file_path)

        self.source_code = source_code.splitlines()

        self.alias_dict = {}
        # self.variable_types = {}
        self.analyzed_files = set()
        self.indent_level = 0
        
        self.file_analyzers = {}
        # self.function_defs = {}
        # self.current_function_stack = []
        # self.global_function_defs = global_function_defs

    def get_node_content(self, node):
        start_line = node.lineno - 1
        end_line = node.end_lineno
        content = '\n'.join(self.source_code[start_line:end_line]).strip()
        return content

    def visit_Import(self, node):
        content = self.get_node_content(node)
        for alias in node.names:
            tmp_local_file_path = alias.name.replace('.', os.path.sep) + ".py"
            tmp_abs_file_path = os.path.join(self.folder_path, tmp_local_file_path)

            local_file = os.path.exists(tmp_abs_file_path)

            if local_file and tmp_abs_file_path not in self.analyzed_files:
                self.analyzed_files.add(tmp_abs_file_path)
                root = analyze_python_file(tmp_abs_file_path, self.save_folder)
                self.file_analyzers[tmp_abs_file_path] = root

            new_alias = Alias(actual_name=alias.name, alias_name=alias.asname or alias.name, local_file=local_file)
            self.alias_dict[alias.asname or alias.name] = new_alias

            import_node = Node(
                type="import",
                content=content,
                line_number=node.lineno,
                end_line_number=node.end_lineno,
                alias=new_alias,
                indent_level=self.indent_level
            )
            self.current_node.children.append(import_node)

    def visit_ImportFrom(self, node):
        content = self.get_node_content(node)
        module = node.module or ""
        
        tmp_local_file_path = module.replace('.', os.path.sep) + ".py"
        tmp_abs_file_path = os.path.join(self.folder_path, tmp_local_file_path)

        local_file = os.path.exists(tmp_abs_file_path)

        if local_file and tmp_abs_file_path not in self.analyzed_files:
            self.analyzed_files.add(tmp_abs_file_path)
            root = analyze_python_file(tmp_abs_file_path, self.save_folder)
            self.file_analyzers[tmp_abs_file_path] = root


        if len(node.names) == 1 and node.names[0].name == '*':
            self.handle_star_import(module, tmp_abs_file_path if local_file else None)
        else:
            for alias in node.names:
                new_alias = Alias(
                    actual_name=f"{module}.{alias.name}", 
                    alias_name=alias.asname or alias.name, 
                    local_file=local_file
                )
                self.alias_dict[alias.asname or alias.name] = new_alias

                import_node = Node(
                    type="import",
                    content=content,
                    line_number=node.lineno,
                    end_line_number=node.end_lineno,
                    alias=new_alias,
                    indent_level=self.indent_level
                )
                self.current_node.children.append(import_node)

    def handle_star_import(self, module_name, module_path):
        if module_path:
            with open(module_path, 'r') as file:
                module_ast = ast.parse(file.read())
            names = self.get_module_names(module_ast)
        else:
            try:
                module = importlib.import_module(module_name)
                names = dir(module)
            except ImportError:
                print(f"Warning: Unable to import module {module_name}")
                return

        for name in names:
            if not name.startswith('_'):
                new_alias = Alias(
                    actual_name=f"{module_name}.{name}",
                    alias_name=name,
                    local_file=bool(module_path)
                )
                self.alias_dict[name] = new_alias

    def get_module_names(self, module_ast):
        names = []
        for node in module_ast.body:
            if isinstance(node, (ast.FunctionDef, ast.ClassDef)):
                names.append(node.name)
        return names
